## The sole purpose of this notebook is to comprehend the addition of user profiles to the data set

In [1]:
import pandas as pd

In [142]:
user_profiles = pd.read_csv('data/tweets/TwitterUserClassification.csv')
df = pd.read_csv('data/tweets/IchBinHanna.csv')
#get the correct data
#df= df[['author.description', 'author.id', 'author.name', 'author.public_metrics.followers_count', 'author.public_metrics.following_count', 'author.username', 'author_id', 'created_at', 'id', 'lang', 'public_metrics.retweet_count','text', 'hashtags','reference_type','wanted_tag']]
df['new_date'] = pd.to_datetime(df['created_at']).dt.strftime('%Y-%m-%d %H:%M:%S')
df = df.loc[(df['new_date'] > '2021-06-01 00:00:00') & (df['new_date'] <= '2021-09-30 23:59:59')]
#df= df[['author.description', 'author.id', 'author.name', 'author.public_metrics.followers_count', 'author.public_metrics.following_count', 'author.username', 'author_id', 'created_at', 'id', 'lang', 'public_metrics.retweet_count','text', 'hashtags','reference_type','wanted_tag']]
#df = df.loc[df['reference_type'] != 'retweeted']
df = df[df['text'].str.contains("#ichbinhanna", case = False)]
#df =df.loc[df['wanted_tag']== True]
df = df.loc[(df['lang'] == 'de') | (df['lang'] == 'en')]

C:\Users\Admin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (6,8,9,10,13,37,41,42,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [143]:
#total number of tweets covered by the user profiles
user_profiles['totaltweetsbyauthor'].sum()

99669

In [144]:
user_profiles

,author.id,author.username,author.name,author.description,user.group,totaltweetsbyauthor
0,1.718151e+08,thothiel,Thorsten Thiel,"Politikwissenschaft, Politische Theorie, Wiss...",Academic unspecified,25
1,7.189508e+17,jtetza,Arbeitsrecht,Jeden Tag ein Tweet zum Arbeitsrecht.\nPräsent...,NaN,4
2,2.609103e+09,yrlaNor,@yrlaNor@social.tchncs.de ☀️,wahrheit ist arbeit. @angstalt @C3S @RKWardNet...,NaN,21
3,3.024848e+09,hrnschrttmchr,Jan,hier privat,NaN,11
4,2.565771e+09,Internaturalbaw,Internaturals,Pop Culture|Nature|Travel \n\nhttps://t.co/hyi...,NaN,11
...,...,...,...,...,...,...
15747,1.248565e+18,hmheinig,@hmheinig,Öffentliches Recht & Kirchenrecht + dies & das...,NaN,1
15748,3.096281e+09,jkflss,Joke F.,cahiers in Zeiten von Social Media.,NaN,2
15749,1.372144e+09,marius_s_klug,Marius Klug,"PhD student at BeMoBIL of @KlausGramann, resea...",Promovierende,1
15750,1.428095e+18,Agustin69501103,Agustina Carrizo,Always looking for the right question,NaN,1


In [145]:
df['author.id'] = df['author.id'].astype(int)
user_profiles['author.id'] = user_profiles['author.id'].astype(int)

In [146]:
df['author.id']

0        -2147483648
1        -2147483648
3        -2147483648
4        -2147483648
6        -2147483648
             ...    
116922   -2147483648
116923     329505955
116924   -2147483648
116925   -2147483648
116927   -2147483648
Name: author.id, Length: 60832, dtype: int32

In [147]:
#unique authors in IchBinHanna.csv
len(pd.unique(df['author.id']))

4632

In [148]:
len(user_profiles)

15752

In [149]:
#unique classified users
len(pd.unique(user_profiles['author.id']))

5463

In [150]:
user_merge = user_profiles[['author.username', 'user.group']].drop_duplicates(subset= 'author.username')

In [151]:
len(user_merge)

15643

In [152]:
#add profile info to tweets
merged_df = df.merge(user_merge )

In [153]:
#remaining tweets
len(merged_df)

60755

In [154]:
#initial tweets
len(df)

60832

Joining on the unique user ID should classify all users, but that is not the case, because the conversion of the IDs in the data set that was used to build the user profiles in R cut parts of the IDs off, as they were probably too long for the chosen format. Joining on username gave a better result, as it is also unique and can't usually be changed (however there are a few exceptions). Those exceptions are added by hand 

In [155]:
#distribution of user profiles in combined data frame
test = merged_df['user.group'].value_counts()
test

Postdoc                 18317
Academic unspecified    10173
Promovierende            3743
Prof                     3119
Bot                      1433
Media                    1131
Student                   940
Institution               689
Union rep                 556
Political rep             368
Jun. prof                 320
Teacher                   143
Medical doctor              5
Name: user.group, dtype: int64

In [156]:
unique_merged = merged_df.drop_duplicates(subset='author.name', keep="last")

In [157]:
#distribution of unique users in combined data frame
unique_merged['user.group'].value_counts()

Academic unspecified    2021
Postdoc                 1209
Promovierende            798
Prof                     639
Media                    297
Student                  283
Political rep            114
Teacher                   81
Institution               60
Union rep                 52
Jun. prof                 35
Bot                       25
Medical doctor             3
Name: user.group, dtype: int64

In [158]:
#distribution in user profile data frame
user_profiles['user.group'].value_counts()

Academic unspecified    2392
Postdoc                 1376
Promovierende            955
Prof                     716
Student                  339
Media                    331
Political rep            132
Teacher                   96
Institution               76
Union rep                 70
Jun. prof                 38
Bot                       29
Medical doctor             3
Name: user.group, dtype: int64

In [159]:
#identify users that are unclassified
diff = pd.concat([df,merged_df]).drop_duplicates(subset= 'id',keep = False)

In [160]:
#number of authors that have to be classified by hand
len(pd.unique(diff['author_id']))

12

In [161]:
for i in user_profiles['author.description'].loc[user_profiles['author.username'] == "Duonna_Antonia"]:
    print(i)

Schreibt – recherchiert – plaudert, am liebsten über Menschenrechte und Politik im 17. oder 21. Jahrhundert.\nBuch: Bergünerstein I. Der Krieg\n#Wähencontent
Schreibt – recherchiert – plaudert, am liebsten über Menschenrechte und Politik im 17. oder 21. Jahrhundert.\nBuch: Bergünerstein I. Der Krieg\n#Wähencontent


In [162]:
diff

,__twarc.retrieved_at,__twarc.url,__twarc.version,attachments.media,attachments.media_keys,attachments.poll.duration_minutes,attachments.poll.end_datetime,attachments.poll.id,attachments.poll.options,attachments.poll.voting_status,...,text,type,withheld.scope,withheld.copyright,withheld.country_codes,hashtags,reference_type,wanted_tag,new_date,user.group
93303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,In jeder Superhelden Story gibt es auch immer ...,retweeted,NaN,NaN,NaN,['IchbinHanna'],quoted,True,2021-09-26 18:31:09,NaN
93602,2021-10-01T06:36:53+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Diese #IchBinHanna war diese Woche mental an a...,NaN,NaN,NaN,NaN,['IchBinHanna'],no_reference,True,2021-09-25 18:38:44,NaN
93611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mal schauen welche Partei #afd diese fleißigen...,replied_to,NaN,NaN,NaN,"['afd', 'Krankenschwestern', 'pflege', 'btw21'...",replied_to,True,2021-09-25 03:44:26,NaN
93612,2021-10-01T06:36:53+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mal schauen welche Partei #afd diese fleißigen...,NaN,NaN,NaN,NaN,"['afd', 'Krankenschwestern', 'pflege', 'btw21'...",replied_to,True,2021-09-25 16:19:19,NaN
93613,2021-10-01T06:36:53+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Mal schauen welche Partei #afd diese fleißigen...,NaN,NaN,NaN,NaN,"['afd', 'Krankenschwestern', 'pflege', 'btw21'...",replied_to,True,2021-09-25 16:18:47,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116805,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Typisch akademisches gegendertes linksgrünes G...,NaN,NaN,NaN,NaN,['IchBinHanna'],no_reference,True,2021-07-27 09:34:33,NaN
116807,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Typisch akademisches gegendertes linksgrünes G...,NaN,NaN,NaN,NaN,['IchBinHannah'],no_reference,True,2021-07-27 09:33:58,NaN
116819,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Eine Arroganz hier,für studierte brotlose Kuns...",NaN,NaN,NaN,NaN,['IchBinHannah'],no_reference,True,2021-07-27 09:30:41,NaN
116823,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Eine Arroganz hier,für studierte brotlose Kuns...",NaN,NaN,NaN,NaN,['IchbinHanna'],no_reference,True,2021-07-27 09:29:39,NaN


In [163]:
test = diff[['author.id','author.username', 'author.name']]

In [164]:
final_df = pd.concat([merged_df,diff])

In [165]:
final_df

,__twarc.retrieved_at,__twarc.url,__twarc.version,attachments.media,attachments.media_keys,attachments.poll.duration_minutes,attachments.poll.end_datetime,attachments.poll.id,attachments.poll.options,attachments.poll.voting_status,...,text,type,withheld.scope,withheld.copyright,withheld.country_codes,hashtags,reference_type,wanted_tag,new_date,user.group
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Summer break. Pause, leisure, relaxation. Mayb...",retweeted,NaN,NaN,NaN,['IchbinHanna'],no_reference,True,2021-07-26 21:40:18,Academic unspecified
1,2021-07-27T13:10:02+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Hanna will be just another number in the acade...,NaN,NaN,NaN,NaN,"['WirsindHanna', 'IchbinHanna']",replied_to,True,2021-07-26 21:40:23,Academic unspecified
2,2021-07-27T13:10:22+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,RT @schuh_melissa: Wichtiger Beitrag von der w...,NaN,NaN,NaN,NaN,"['IchBinHanna', 'IchBinReyhan']",retweeted,True,2021-07-22 13:50:02,Academic unspecified
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Being #Hanna is, for many, a permanent, irreme...",retweeted,NaN,NaN,NaN,"['Hanna', 'IchBinHanna', 'WirsindHanna']",quoted,True,2021-07-16 12:01:42,Academic unspecified
4,2021-07-27T13:11:39+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,RT @Schaerferin: @mahaelhissy stellt die entsc...,NaN,NaN,NaN,NaN,['IchBinHanna'],retweeted,True,2021-07-12 13:52:51,Academic unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116805,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Typisch akademisches gegendertes linksgrünes G...,NaN,NaN,NaN,NaN,['IchBinHanna'],no_reference,True,2021-07-27 09:34:33,NaN
116807,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Typisch akademisches gegendertes linksgrünes G...,NaN,NaN,NaN,NaN,['IchBinHannah'],no_reference,True,2021-07-27 09:33:58,NaN
116819,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Eine Arroganz hier,für studierte brotlose Kuns...",NaN,NaN,NaN,NaN,['IchBinHannah'],no_reference,True,2021-07-27 09:30:41,NaN
116823,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Eine Arroganz hier,für studierte brotlose Kuns...",NaN,NaN,NaN,NaN,['IchbinHanna'],no_reference,True,2021-07-27 09:29:39,NaN


In [166]:
print(pd.unique(diff['author.username']))

['M_Sc_ING' 'jnnfrschr' 'Kommsemit' 'angelag_94' 'kingotnik' 'chr_riedel'
 'JanOliverJost' 'mkudlowski' 'FrlThiel' 'HetEconVienna' 'heinekenuwe'
 'ProfCha0s84']


In [167]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(test)

         author.id author.username       author.name
93303  -2147483648        M_Sc_ING             M.Sc.
93602  -2147483648       jnnfrschr  Jennifer Schauer
93611  -2147483648       Kommsemit          Trapezit
93612  -2147483648       Kommsemit          Trapezit
93613  -2147483648       Kommsemit          Trapezit
93614  -2147483648       Kommsemit          Trapezit
93743  -2147483648       Kommsemit          Trapezit
93744  -2147483648       Kommsemit          Trapezit
93745  -2147483648       Kommsemit          Trapezit
93777  -2147483648       Kommsemit          Trapezit
93877  -2147483648        M_Sc_ING             M.Sc.
93996  -2147483648       Kommsemit          Trapezit
94677  -2147483648        M_Sc_ING             M.Sc.
94905  -2147483648      angelag_94            Angela
95089  -2147483648      angelag_94            Angela
95641  -2147483648       Kommsemit          Trapezit
95642  -2147483648       Kommsemit          Trapezit
96246  -2147483648       jnnfrschr  Jennifer S

In [168]:
user_profiles.loc[user_profiles['author.name'] == 'Jan Oliver Jost']

,author.id,author.username,author.name,author.description,user.group,totaltweetsbyauthor
11142,-2147483648,7d88461a83e64f6,Jan Oliver Jost,NaN,NaN,8


In [169]:
tester = user_profiles[user_profiles['author.description'].notna()]
tester[tester['author.name'].str.contains("Es war einmal")]

,author.id,author.username,author.name,author.description,user.group,totaltweetsbyauthor


In [170]:
pd.options.display.max_colwidth = 50
final_df['author.description'].loc[final_df['author.name'] == 'Prof Chaos']

112304    Es war ein Mal...\n\nEhemals etwas mit Pandas
Name: author.description, dtype: object

In [171]:
final_df['user.group'].loc[final_df['author.username'] == 'kingotnik'] = 'Postdoc'
final_df['user.group'].loc[final_df['author.username'] == 'jnnfrschr'] = 'Promovierende'
final_df['user.group'].loc[final_df['author.username'] == 'chr_riedel'] = 'Postdoc'
final_df['user.group'].loc[final_df['author.username'] == 'mkudlowski'] = 'Academic unspecified'
final_df['user.group'].loc[final_df['author.username'] == 'angelag_94'] = 'Academic unspecified'
final_df['user.group'].loc[final_df['author.username'] == 'FrlThiel'] = 'Academic unspecified'

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [172]:
final_df

,__twarc.retrieved_at,__twarc.url,__twarc.version,attachments.media,attachments.media_keys,attachments.poll.duration_minutes,attachments.poll.end_datetime,attachments.poll.id,attachments.poll.options,attachments.poll.voting_status,...,text,type,withheld.scope,withheld.copyright,withheld.country_codes,hashtags,reference_type,wanted_tag,new_date,user.group
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Summer break. Pause, leisure, relaxation. Mayb...",retweeted,NaN,NaN,NaN,['IchbinHanna'],no_reference,True,2021-07-26 21:40:18,Academic unspecified
1,2021-07-27T13:10:02+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Hanna will be just another number in the acade...,NaN,NaN,NaN,NaN,"['WirsindHanna', 'IchbinHanna']",replied_to,True,2021-07-26 21:40:23,Academic unspecified
2,2021-07-27T13:10:22+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,RT @schuh_melissa: Wichtiger Beitrag von der w...,NaN,NaN,NaN,NaN,"['IchBinHanna', 'IchBinReyhan']",retweeted,True,2021-07-22 13:50:02,Academic unspecified
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Being #Hanna is, for many, a permanent, irreme...",retweeted,NaN,NaN,NaN,"['Hanna', 'IchBinHanna', 'WirsindHanna']",quoted,True,2021-07-16 12:01:42,Academic unspecified
4,2021-07-27T13:11:39+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,RT @Schaerferin: @mahaelhissy stellt die entsc...,NaN,NaN,NaN,NaN,['IchBinHanna'],retweeted,True,2021-07-12 13:52:51,Academic unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116805,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Typisch akademisches gegendertes linksgrünes G...,NaN,NaN,NaN,NaN,['IchBinHanna'],no_reference,True,2021-07-27 09:34:33,NaN
116807,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Typisch akademisches gegendertes linksgrünes G...,NaN,NaN,NaN,NaN,['IchBinHannah'],no_reference,True,2021-07-27 09:33:58,NaN
116819,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Eine Arroganz hier,für studierte brotlose Kuns...",NaN,NaN,NaN,NaN,['IchBinHannah'],no_reference,True,2021-07-27 09:30:41,NaN
116823,2021-10-01T06:52:57+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.3.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Eine Arroganz hier,für studierte brotlose Kuns...",NaN,NaN,NaN,NaN,['IchbinHanna'],no_reference,True,2021-07-27 09:29:39,NaN


In [173]:
#write df with user profiles to csv and use that csv for the analysis
compression_opts = dict(method='zip',
                        archive_name='IchBinHannaUser.csv')  
final_df.to_csv('out.zip', index=False,
          compression=compression_opts) 